In [1]:
import pybamm as pb;import pandas as pd   ;import numpy as np;import os;import matplotlib.pyplot as plt;import os;#import imageio
from scipy.io import savemat,loadmat;from pybamm import constants,exp;import matplotlib as mpl; fs=17; # or we can set import matplotlib.pyplot as plt then say 'mpl.rc...'
for k in range(0,1):
    mpl.rcParams["axes.labelsize"] = fs
    mpl.rcParams["axes.titlesize"] = fs
    mpl.rcParams["xtick.labelsize"] =  fs
    mpl.rcParams["ytick.labelsize"] =  fs
    mpl.rcParams["legend.fontsize"] =  fs
    mpl.rcParams['font.sans-serif'] = ['Times New Roman']
    mpl.rcParams['font.serif'] = ['Times New Roman']
    mpl.rcParams['axes.unicode_minus'] = False
import openpyxl
import traceback
import multiprocessing
import scipy.optimize

In [2]:
Chemistry=pb.parameter_sets.OKane2022 # Chen2020 # 
#Chemistry["electrolyte"] = "lipf6_Nyman2008"
#Chemistry["negative electrode"] = "graphite_Chen2020"
Chemistry["sei"] = "example"
Para_0=pb.ParameterValues(chemistry=Chemistry)
model_options={
    "calculate discharge energy":"true",
    "particle": "Fickian diffusion",          
    "SEI":"interstitial-diffusion limited",   
    "SEI on cracks":"true",  
    "SEI film resistance":"distributed",          
    "SEI porosity change":"true",      
    "particle mechanics":("swelling and cracking", "swelling only"), 
    "loss of active material":"stress-driven", 
    "lithium plating":"partially reversible"      
}
Model_0 = pb.lithium_ion.DFN(options=model_options ) #
#Para_0.update( {"Inner SEI reaction proportion":0.5})
#Para_0.update( {"Initial inner SEI thickness [m]":2.5E-9})
#Para_0.update( {"Initial outer SEI thickness [m]":2.5E-9})
#Para_0.update( {"Ratio of lithium moles to SEI moles":2})
""" Neg1SOC_in = 0.850;Pos1SOC_in=0.270
c_Neg1SOC_in = (
    Para_0["Maximum concentration in negative electrode [mol.m-3]"]
    *Neg1SOC_in)
c_Pos1SOC_in = (
    Para_0["Maximum concentration in positive electrode [mol.m-3]"]
    *Pos1SOC_in)
Para_0.update(
    {"Initial concentration in negative electrode [mol.m-3]":
    c_Neg1SOC_in})
Para_0.update(
    {"Initial concentration in positive electrode [mol.m-3]":
    c_Pos1SOC_in}) 
Para_0.update(
    {"Negative electrode cracking rate":
    3.9e-25}) 
Para_0.update(
    {"Positive electrode cracking rate":
    3.9e-25})  """
var = pb.standard_spatial_vars  
var_pts = {
    var.x_n: 20,  
    var.x_s: 10,  
    var.x_p: 20,  
    var.r_n: int(30),  
    var.r_p: int(30),  }       
submesh_types = Model_0.default_submesh_types
particle_mesh = pb.MeshGenerator(
    pb.Exponential1DSubMesh, 
    submesh_params={"side": "right", "stretch": 1.5})
submesh_types["negative particle"] = particle_mesh
submesh_types["positive particle"] = particle_mesh 
V_max = 4.2;        
V_min = 2.5; 
exp_RPT_text = [ (
    f"Discharge at 0.1C until {V_min} V",  
    "Rest for 1 hours",  
    f"Charge at 0.1C until {V_max} V" ) ] # (5 minute period)
exp_RPT_s = [ (
    f"Discharge at 0.1C for 10 s",  ) ] 
RPT_Cycles=1
Experiment_RPT    = pb.Experiment( exp_RPT_text * RPT_Cycles     ) 
Experiment_s    = pb.Experiment( exp_RPT_s * RPT_Cycles     ) 
Sim_0    = pb.Simulation(
    Model_0,        experiment = Experiment_RPT,
    parameter_values = Para_0,
    solver = pb.CasadiSolver(),
    var_pts=var_pts,
    submesh_types=submesh_types
    ) #mode="safe"
Sol_0    = Sim_0.solve(calc_esoh=False)

In [3]:
# Quick plot
label = ["Sol_0"] 
output_variables3 = [
    "Terminal voltage [V]",
    "Negative electrode capacity [A.h]",
    "Positive electrode capacity [A.h]",
    "Loss of capacity to lithium plating [A.h]",
    "Loss of capacity to SEI [A.h]",
    "Loss of capacity to SEI on cracks [A.h]",
    "Negative electrode SOC",
    "Positive electrode SOC",
    
]
quick_plot = pb.QuickPlot([Sol_0], output_variables3,label,variable_limits='fixed') #     
quick_plot.dynamic_plot();

interactive(children=(FloatSlider(value=0.0, description='t', max=21.592712580266888, step=0.21592712580266887…